# Homework 3: Logistic Regression and Stochastic Gradient Descent



This assignment is due on Canvas by **11:59pm on Sunday April 23**. 
Your solutions to theoretical questions should be done in Markdown/MathJax directly below the associated question.
Your solutions to computational questions should include any specified Python code and results 
as well as written commentary on your conclusions.
Remember that you are encouraged to discuss the problems with your instructors and classmates, 
but **you must write all code and solutions on your own**. For a refresher on the course **Collaboration Policy** click [here](https://canvas.uchicago.edu/courses/49007).

**NOTES**: 

- Some problems with code may be autograded.  If we provide a function API, **do not** change it.  If we do not provide a function API, then you're free to structure your code however you like. 
- Submit this Jupyter notebook to Canvas.  Do not compress it using tar, rar, zip, etc.
- Also submit a PDF of this Jupyter notebook to Canvas.

**Acknowledgment**: Noah Smith, Chris Ketelsen


**Name**:

In [1]:
# Import required packages
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline

## [100 points] Logistic Regression + SGD

In this assignment, you'll implement a Logistic Regression classifier to predict whether a reported crime (incident) results in an arrest.  We use the Chicago crime dataset used in Assignment 1, with a few modifications described below.  


Dataset has following attributes:

|Variable|Definition|Key|
|:----:|:----:|:---|
|Hour|Time of incident|integer|
|Domestic|Category of Crime|bool|
|Primary Type |Type of Crime|one hot encoded (all types with moderate arrest rate)|
|Ward	|Location of incident|one hot encoded|
|Arrest	|Whether an arrest was made|bool|




The following cell is a class to load the crime dataset.

In [2]:
class Dataset:
    """
    Class to load dataset containing Chicago crime features
    You shouldn't have to modify this class.
    """

    def __init__(self, location, random_state=1241):
        # Load the dataset
        np.random.seed(random_state)
        small_df = pd.read_csv(location)
        y_crime_df, X_crime_df  = small_df[['Arrest']], small_df.drop(['Arrest'], axis=1)
        self.train_x, self.test_x, self.train_y, self.test_y = train_test_split(
            X_crime_df.to_numpy(), y_crime_df.to_numpy(), test_size=0.2, random_state=123)
        
        # appending biases
        self.train_x = np.concatenate((np.ones((self.train_x.shape[0], 1)), self.train_x), axis=1)
        self.test_x = np.concatenate((np.ones((self.test_x.shape[0], 1)), self.test_x), axis=1)
        

    @staticmethod
    def shuffle(X, y):
        """ Shuffle training data """
        shuffled_indices = np.random.permutation(len(y))
        return X[shuffled_indices], y[shuffled_indices]

### Part 1 [10 points]: Implementing sigmoid

#### Part 1 A [7 points] 
First, implement the `sigmoid` function to return the output by applying the sigmoid function $\sigma(z)$ to the input parameter, where the sigmoid function $\sigma(z)$ is defined as:
$$
\sigma(z) = \frac{1}{1+e^{-z}}
$$

In [25]:
def sigmoid(score, threshold=20.0):
    """
    Sigmoid function with a threshold
    :param score: (float) A real valued number to convert into a number between 0 and 1 
    :param threshold: (float) Prevent overflow of exp by capping activation at 20. 
                    (e.g., scores higher than 20 are converted to 20, scores lower than -20 are converted to -20).                 
    
    :return: (float) sigmoid function result.
    """
    if score < -threshold:
        score = -threshold
    elif score > threshold:
        score = threshold
    sigm = 1 / (1 + math.exp(-score))
    return sigm


In [26]:
# verify sigmoid implemention w/ scipy;
# note: you should NOT use scipy for your implementation!
from scipy.stats import logistic
assert sigmoid(1) == logistic.cdf(1)
assert sigmoid(5) == logistic.cdf(5)
assert sigmoid(100, threshold=20) == logistic.cdf(20)
assert sigmoid(-1) == logistic.cdf(-1)
assert sigmoid(-5) == logistic.cdf(-5)
assert sigmoid(-100, threshold=20) == logistic.cdf(-20)

0.9999999979388463
0.9999999979388463


#### Part 1 B [3 points]

Next, implement the derivative of the `sigmoid` function, `sigmoid_grad`, i.e. $\frac{\partial\sigma(x)}{\partial x}$.

Hint: your implementation of `sigmoid_grad` should be able to use  your `sigmoid` function to compute the derivative!

In [33]:
def sigmoid_grad(y, threshold=20.0):
    """
    Derivative/gradient of the sigmoid function.
    :param y: (float) A real valued input for which to compute the derivative.
    :param threshold: (float) Prevent overflow of exp by capping activation at 20.
    
    :return: (float) sigmoid derivative function result.
    """
    grad = sigmoid(y, threshold) * (1 - sigmoid(y, threshold))
    return grad
    

In [34]:
# verify sigmoid_grad using numerical differentiation, i.e: f(x+h)-f(x-h) / 2h
epsilon = 1.0E-8
assert np.isclose(sigmoid_grad(1.0), (sigmoid(1.0 + epsilon) - sigmoid(1.0 - epsilon)) / (2.0*epsilon))
assert np.isclose(sigmoid_grad(0.1), (sigmoid(0.1 + epsilon) - sigmoid(0.1 - epsilon)) / (2.0*epsilon))

### Part 2 [80 points]

#### Part 2 A [20 points]

The negative log likelihood objective is defined as:
$$
\textrm{NLL}(\boldsymbol{\beta}) = -\displaystyle\sum_{i=1}^n \left[y_i \log \sigma(\boldsymbol{\beta}^T{\bf x}^{(i)}) + (1-y_i)\log(1 - \sigma(\boldsymbol{\beta}^T{\bf x}^{(i)}))\right] 
$$

First, write down the derivative of the negative log likelihood objective function, with respect to $\boldsymbol{\beta}$. Since we are working with SGD, derive it for  $n=1$.

YOUR ANSWER HERE

Next, using the `sigmoid` function implemented earlier, finish the `sgd_update` function so that it performs stochastic gradient descent on the single training example and updates the weight vector correspondingly without regularization.

In [35]:
class LogReg:
    def __init__(self, num_features, eta):
        """
        Create a logistic regression classifier
        :param num_features: (int) The number of features (including bias)
        :param eta: (float) learning rate
        """
        self.w = np.zeros(num_features)
        self.eta = eta

    def progress(self, examples_x, examples_y):
        """
        Given a set of examples, compute the probability and accuracy
        :param examples_x: (2D np.ndarray) The features from the dataset to score
        :param examples_y: (1D np.ndarray) The labels from the dataset to score

        :return: (float, float) A tuple of (log probability, accuracy)
        """

        logprob = 0.0
        num_right = 0
        for x_i, y in zip(examples_x, examples_y):
            p = sigmoid(self.w.dot(x_i))
            if y == 1:
                logprob += math.log(p)
            else:
                logprob += math.log(1.0 - p)

            # Get accuracy
            if abs(y - p) <= 0.5:
                num_right += 1

        return logprob, float(num_right) / float(len(examples_y))

    def sgd_update(self, x_i, y, lam=0.0):
        """
        Compute a stochastic gradient update to improve the log likelihood.
        :param x_i: (1D np.ndarray) The features of the example to take the gradient with respect to
        :param y: (float) The target output of the example to take the gradient with respect to
        :param lam: (float) regularization term. Default is zero; only used in Part 2D.
        
        :return: (1D np.ndarray) Return the new value of the regression coefficients
        """

        # TODO: Finish this function to do a single stochastic gradient descent update

        # YOUR CODE HERE
        raise NotImplementedError()
        return self.w
        

In [36]:
from tests import tests
tests.run_test_suite('prob 2A', LogReg)

testPosUnregUpdate (tests.tests.TestLogReg)
test update based on positive example ... ERROR
testNegUnregUpdate (tests.tests.TestLogReg)
test update based on negative example ... ERROR

ERROR: testPosUnregUpdate (tests.tests.TestLogReg)
test update based on positive example
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/home/yujie0706/Machine-Learning-Homework/hw3/tests/tests.py", line 26, in testPosUnregUpdate
    lr.sgd_update(train_x[i],train_y[i], lam=0)
  File "/tmp/user/23071/ipykernel_1596183/2969371505.py", line 48, in sgd_update
    raise NotImplementedError()
NotImplementedError

ERROR: testNegUnregUpdate (tests.tests.TestLogReg)
test update based on negative example
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/home/yujie0706/Machine-Learning-Homework/hw3/tests/tests.py", line 41, in testNegUnregUpdate
    lr.sgd_update(train_x[i],train_y[i]

AssertionError: one or more tests for prob 2A failed

#### Part 2 B [20 points]
Complete the code below to loop over the training data and perform stochastic gradient descent for a pre-defined number of epochs. You do not need to use the parameters lam and decay for this part.

Note: remember to shuffle your training data using `Dataset.shuffle` at the beginning of each epoch.

In [ ]:
def train(epochs, eta, store_epoch, lam=0, decay=0):
    """
    Train a LogReg object for a set number of epochs with a given eta.
    
    :param epochs: (int) total number of training epochs
    :param eta: (float) learning rate
    :param store_epoch: (int) store training and test accuracies every store_epoch epochs
    :param lam: (float) weight given to regularization term. Default 0. Only used in Part 2D. 
    :param decay: (float) Used to update learning rate during training (Part 3). 
                  Equals 0 when learning rate is constant throughout training (Part 2). 
                  
    :return (train_accuracy_array, test_accuracy_array, learning_rates): tuple of (List, List, List)
        :train_accuracy_array: training accuracy after every store_epoch epochs
        :test_accuracy_array: test accuracy after every store_epoch epochs
        :learning_rates: learning rate after every store_epoch epochs. All values in this list 
                         will be the same if decay = 0 (Only required for Part 2F)
    
        Example: With epochs = 30 and store_epoch = 10, only store accuracies after epochs = 10, 20, and 30.
    """
    
    dataset_handler = Dataset('data/crime.csv')
    lr = LogReg(dataset_handler.train_x.shape[1], eta)

    assert dataset_handler.train_x.shape == (1105, 60) 
    assert dataset_handler.test_x.shape == (277, 60) 
    
    train_accuracy_array = []
    test_accuracy_array = []
    learning_rates = []
    for epoch in range(epochs):
        # TODO: Finish the code to loop over the training data and perform a stochastic
        # gradient descent update on each training example.

        # NOTE: It may be helpful to call upon the 'progress' method in the LogReg class
        # to make sure the algorithm is truly learning properly on both training and test data
        
    # YOUR CODE HERE
    raise NotImplementedError()
    return train_accuracy_array, test_accuracy_array, learning_rates

In [ ]:
eta  = 1e-4
epochs = 300
store_epoch = 50
train_acc, test_acc, _ = train(epochs, eta, store_epoch)

for i in range(len(train_acc)):
    print("\ntrain accuracy after {} epochs: {}".format((i+1)*store_epoch, train_acc[i]))
    print("test accuracy after {} epochs: {}".format((i+1)*store_epoch, test_acc[i]))


#### Part 2 C [10 points]
What is the role of the learning rate? What are the pros and cons of high/low learning rates? Do you see any trade-off? First, plot accuracies of different $\eta$s together vs. number of epochs for both training and testing. Then briefly elaborate on these questions.

In [ ]:
dataset_handler = Dataset('data/crime.csv')
train_results = {}
test_results = {}

epochs = 400
store_epoch = 10
for eta in [1e-3, 1e-4, 1e-5, 1e-6]:

    # TODO: 
    # Finish the code to loop over different values of learning rates (Use the train() function above)
    
    # You need to store accuracy arrays obtained in the dictionaries provided 
    # above (train_results and test_results)
    
    # Effectively, you will be creating a mapping between eta -> train/test_accuracy_array 
    # Therefore, running train_results[eta] should return the train_accuracy_array for that value
    # of eta and likewise for test_results[eta].
    
    # YOUR CODE HERE
    raise NotImplementedError()

Plot training results below.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12,6))
epochs_array = [i for i in range(store_epoch, epochs, store_epoch)]
epochs_array.append(epochs)
ax.plot(epochs_array, train_results[1e-3], color="steelblue", label='1e-03')
ax.plot(epochs_array, train_results[1e-4], color="lightblue", label='1e-04')
ax.plot(epochs_array, train_results[1e-5], color="grey", label='1e-05')
ax.plot(epochs_array, train_results[1e-6], color="black", label='1e-06')
ax.legend(loc="lower right", fontsize=16)
ax.set_xlabel("epochs", fontsize=16)
ax.set_ylabel("train accuracy", fontsize=16)
plt.show()

Plot testing results below.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12,6))
epochs_array = [i for i in range(store_epoch,epochs,store_epoch)]
epochs_array.append(epochs)
ax.plot(epochs_array, test_results[1e-3], color="steelblue", label='1e-03')
ax.plot(epochs_array, test_results[1e-4], color="lightblue", label='1e-04')
ax.plot(epochs_array, test_results[1e-5], color="grey", label='1e-05')
ax.plot(epochs_array, test_results[1e-6], color="black", label='1e-06')
ax.legend(loc="lower right", fontsize=16)
ax.set_xlabel("epochs", fontsize=16)
ax.set_ylabel("test accuracy", fontsize=16)
plt.show()

YOUR ANSWER HERE

#### Part 2 D [15 points]

Adding $l_2$ regularization to the feature parameters for NLL loss gives:

$$
\textrm{NLL}_{l_2}(\boldsymbol{\beta}) = -\displaystyle\sum_{i=1}^n \left[y_i \log \sigma(\boldsymbol{\beta}^T{\bf x}^{(i)}) + (1-y_i)\log(1 - \sigma(\boldsymbol{\beta}^T{\bf x}^{(i)}))\right] + \lambda\displaystyle\sum_{k=1}^{p} \beta_{k}^2
$$

where $p$ is the number of features, and $\beta_0$ is the bias term. Notice that $\beta_0$ is not included in the regularization term.

Write down the derivative of the regularized negative log likelihood loss function $\textrm{NLL}_{l_2}$ with respect to $\boldsymbol{\beta}$. Since we are working with SGD, derive it for $n=1$.

YOUR ANSWER HERE

Update your implementation of the `sgd_update` method so that it performs regularized SGD updates of the model parameters to minimize the regularized NLL loss function.

Remember, do **not** regularize the bias parameter $\beta_0$.

Provide train and test accuracy after above change with `lam=1e-5`.

In [ ]:
from tests import tests
tests.run_test_suite('prob 2E', LogReg)

#### Part 2 E [8 points]
Update your implementation of train() to incorporate a regularization term. The change should typically be on only one line in your code.

Plot accuracies of different $\lambda$s together vs. epochs for both training and testing).

In [ ]:
dataset_handler = Dataset('data/crime.csv')
train_results = {}
test_results = {}
epochs = 400
store_epoch = 10
eta = 1e-4
for lam in [0, 0.1, 0.05, 0.01]:
    
    # TODO: 
    # Finish the code to loop over different values of lambda (Use the train() function above)
    
    # You need to store accuracy arrays obtained in the dictionaries provided 
    # above (train_results and test_results)
    
    # Effectively, you will be creating a mapping between lambda -> train/test_accuracy_array 
    # Therefore, running train_results[lam] should return the train_accuracy_array for that value
    # of lam and likewise for test_results[lam].
    
    # YOUR CODE HERE
    raise NotImplementedError()

Plot training results below.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12,6))
epochs_array = [i for i in range(store_epoch,epochs,store_epoch)]
epochs_array.append(epochs)
ax.plot(epochs_array, train_results[0], color="black", label=str(0))
ax.plot(epochs_array, train_results[0.01], color="grey", label=str(0.01))
ax.plot(epochs_array, train_results[0.05], color="lightblue", label=str(0.05))
ax.plot(epochs_array, train_results[0.1], color="steelblue", label=str(0.1))
ax.legend(loc="lower right", fontsize=16)
ax.set_xlabel("epochs", fontsize=16)
ax.set_ylabel("train accuracy", fontsize=16)
plt.show()

Plot testing results below.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12,6))
epochs_array = [i for i in range(store_epoch,epochs,store_epoch)]
epochs_array.append(epochs)
ax.plot(epochs_array, test_results[0], color="black", label=str(0))
ax.plot(epochs_array, test_results[0.01], color="grey", label=str(0.01))
ax.plot(epochs_array, test_results[0.05], color="lightblue", label=str(0.05))
ax.plot(epochs_array, test_results[0.1], color="steelblue", label=str(0.1))
ax.legend(loc="lower right", fontsize=16)
ax.set_xlabel("epochs", fontsize=16)
ax.set_ylabel("test accuracy", fontsize=16)
plt.show()

### Part 2 F [7 points] 
What is the effect of regularization term with respect to accuracy? 

YOUR ANSWER HERE

### Part 3 [10 points] 

Time based Learning Rate is dynamic learning rate given the following equation:

$\textrm{LearningRate} = \eta\, / \,(1 + \textrm{decay} \cdot \textrm{current epoch})$

Train SGD with the dynamic learning rate defined above and follow these instructions:
* Use initial learning rate $\eta = 0.1$.
* Use $\textrm{decay} = 0.001$.
* Update learning rate `lr.eta` every epoch.
* Use $\lambda = 0$ (no regularization)
* Plot train accuracy and learning rate together for each epoch.

The above can be accomplished by changing one line in `train()` from Part 2a. 

In [ ]:
eta  = 1e-1
epochs = 200
store_epoch = 1

# Lists required for plotting
train_accuracy_array = None
learning_rates = None

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12,6))
epochs_array = [i for i in range(1,epochs)]
epochs_array.append(epochs)
ax.plot(epochs_array, train_accuracy_array, color="steelblue", label=str('train accuracy'))
ax.plot(epochs_array, learning_rates,color="grey", label=str('learning rate'))
ax.legend(loc="center right", fontsize=16)
ax.set_xlabel("epochs", fontsize=16)
ax.set_ylabel("", fontsize=16)
plt.show()

### Optional survey.
***

We are always interested in your feedback. At the end of each homework, there is a simple anonymous feedback [survey](https://docs.google.com/forms/d/e/1FAIpQLSefzoYvcweZYAb_OsNnEyMfbdyyTaVFcOJAnEanwrYSGtNpIQ/viewform) to solicit your feedback for how to improve the course.